In [10]:
pip install mediapipe opencv-python numpy

  Using cached mediapipe-0.10.14-cp39-cp39-win_amd64.whl.metadata (9.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached protobuf-4.25.4-cp39-cp39-win_amd64.whl.metadata (541 bytes)
  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl.metadata (60 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pillow-10.4.0-cp39-cp39-win_amd64.whl.metadata (9.3 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   ---------------------------------------- 0.5/50.8 MB 2.1 MB/s eta 0:00:24
    --------------------------------------- 1.0/50.8 MB 2.5 MB/s eta 0:00:20
   - -------------------------------------- 1.6/50.8 MB 2.5 MB/s eta 0:00:20
   - -------------------------------------- 2.4/50.8 MB 2.7 MB/s eta 0:00:19
   -- ------------------------------------- 2.6/50.8 MB 2.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands.
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

# Initialize MediaPipe drawing utilities.
mp_drawing = mp.solutions.drawing_utils

# Specify the path to the input video.
input_video_path = 'tiktok_sos.mp4'
output_video_path = 'output.mp4'

# Define the frame skipping parameter.
skip_frames = 1

# Open the input video file.
cap = cv2.VideoCapture(input_video_path)

if not cap.isOpened():
    print(f"Error: Unable to open video file {input_video_path}")
    exit()

# Get video properties.
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define codec and initialize VideoWriter.
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

frame_count = 0

def get_landmark_positions(hand_landmarks):
    landmarks = {}
    for id, landmark in enumerate(hand_landmarks.landmark):
        landmarks[id] = (landmark.x, landmark.y, landmark.z)
    return landmarks

def recognize_gesture(hand_landmarks):
    landmarks = get_landmark_positions(hand_landmarks)

    # Define landmark positions for gestures.
    thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP]
    thumb_ip = landmarks[mp_hands.HandLandmark.THUMB_IP]
    index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    index_mcp = landmarks[mp_hands.HandLandmark.INDEX_FINGER_MCP]
    middle_tip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    middle_mcp = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
    ring_tip = landmarks[mp_hands.HandLandmark.RING_FINGER_TIP]
    ring_mcp = landmarks[mp_hands.HandLandmark.RING_FINGER_MCP]
    pinky_tip = landmarks[mp_hands.HandLandmark.PINKY_TIP]
    pinky_mcp = landmarks[mp_hands.HandLandmark.PINKY_MCP]  # Corrected here.

    # Gesture recognition logic.
    if thumb_tip[1] < thumb_ip[1] and index_tip[1] < thumb_tip[1] < middle_tip[1] and \
       ring_tip[1] < thumb_tip[1] < pinky_tip[1]:
        return "Thumbs Up"
    elif thumb_tip[1] > thumb_ip[1] and index_tip[1] > thumb_tip[1] > middle_tip[1] and \
         ring_tip[1] > thumb_tip[1] > pinky_tip[1]:
        return "Thumbs Down"
    elif index_tip[1] > middle_tip[1] > ring_tip[1] > pinky_tip[1]:
        return "Fist"
    elif index_tip[1] < middle_tip[1] < ring_tip[1] < pinky_tip[1] and thumb_tip[1] < thumb_ip[1]:
        return "Open Hand"
    elif (index_tip[1] < index_mcp[1] and pinky_tip[1] < pinky_mcp[1] and 
          middle_tip[1] > middle_mcp[1] and ring_tip[1] > ring_mcp[1]):
        return "I Love You"
    # Recognize Closed Hand: All fingers, including the thumb, are below their respective base joints.
    elif (thumb_tip[1] > thumb_ip[1] and
          index_tip[1] > index_mcp[1] and
          middle_tip[1] > middle_mcp[1] and
          ring_tip[1] > ring_mcp[1] and
          pinky_tip[1] > pinky_mcp[1]):
        return "Closed Hand"
    else:
        return "Unknown Gesture"

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("End of video file or error reading the frame.")
        break

    if frame_count % (skip_frames + 1) == 0:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_id, hand_landmarks in enumerate(results.multi_hand_landmarks):
                gesture = recognize_gesture(hand_landmarks)
                print(f"Hand {hand_id + 1}: {gesture}")

                # Draw hand landmarks and connections.
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Overlay the gesture text on the frame.
                cv2.putText(frame, f'Gesture: {gesture}', (10, 30 + hand_id * 40), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    out.write(frame)
    cv2.imshow('MediaPipe Hands', frame)
    if cv2.waitKey(5) & 0xFF == 27:  # Press 'Esc' to exit.
        break

    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()

End of video file or error reading the frame.


In [32]:
import cv2
import mediapipe as mp
import time

# Initialize MediaPipe Hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

# Function to handle alerts
def send_alert_to_emergency_services():
    print("Alert: Emergency services contacted!")

# Function to detect SOS gesture
def detect_sos_gesture(landmarks, state, time_in_state):
    # Define key landmarks for simplicity
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]
    thumb_ip = landmarks[3]  # Thumb interphalangeal joint

    # Open palm: all fingers extended
    is_open_palm = (index_tip.y < landmarks[6].y and
                    middle_tip.y < landmarks[10].y and
                    ring_tip.y < landmarks[14].y and
                    pinky_tip.y < landmarks[18].y and
                    thumb_tip.x > thumb_ip.x)

    # Fist: all fingers closed over thumb
    is_fist = (index_tip.y > landmarks[6].y and
               middle_tip.y > landmarks[10].y and
               ring_tip.y > landmarks[14].y and
               pinky_tip.y > landmarks[18].y)

    # State transitions with time consistency
    current_time = time.time()
    if state == 0 and is_open_palm:
        print("Step 1: Open palm detected.")
        return 1, current_time  # Move to next step
    elif state == 1 and (current_time - time_in_state > 0.5):
        print("Step 2: Waiting for thumb tucked.")
        return 2, current_time  # Move to next step
    elif state == 2 and is_fist and (current_time - time_in_state > 0.5):
        print("Step 3: Fist made detected.")
        return 3, current_time  # Full SOS gesture recognized

    return state, time_in_state

# Load video from file
video_path = "diya_sos.mp4"  # Path to the video file
cap = cv2.VideoCapture(video_path)

# Initial gesture state and time tracking
gesture_state = 0
state_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with the hand recognition model
    result = hands.process(rgb_frame)

    # Check for detected hand landmarks
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Draw landmarks for visual confirmation
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)
            )

            # Extract landmark positions
            landmarks = hand_landmarks.landmark
            
            # Check for SOS gesture with state management
            gesture_state, state_time = detect_sos_gesture(landmarks, gesture_state, state_time)

            # Reset state if gesture sequence completed
            if gesture_state == 3:
                send_alert_to_emergency_services()
                gesture_state = 0  # Reset after alert

    else:
        print("No hand detected in the current frame.")
        gesture_state = 0  # Reset if no hand is detected

    # Show the frame for visual confirmation
    cv2.imshow('Hand Gesture Recognition', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the current frame.
No hand detected in the c